In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  ScrubChannel is running.               /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import time

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])
    
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

updatetemplate = "{\
    \"ok\": true, \
    \"channel\": \"C024BE91L\", \
    \"ts\": \"1401383885.000061\", \
    \"text\": \"Updated text you carefully authored\", \
    \"message\": { \
        \"text\": \"Updated text you carefully authored\", \
        \"user\": \"DGM50H51N\" \
    } }"

/  ScrubChannel is running.               /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
def updateMessageToSlack(slackchannel, ts, text):
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("chat.update", channel=slackchannel, ts=ts, text=text, blocks=[])
    #print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
      return 'failed'
    else:
      print("Ok posting message to Slack channel")
      return 'ok'

In [3]:
def getMessageList():
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("search.messages", query="KanjiDB", count=100)
    if not 'ok' in response or not response['ok']:      
      return None
    else:
      print("Ok searching Slack workspace.")
      return response['messages']

In [4]:
def deleteMessage(channelid, ts):
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("chat.delete", channel=channelid, ts=ts)
    if not 'ok' in response or not response['ok']:      
      logger(_LOG_ERROR, "error deleting message.")
      logger(_LOG_ERROR, response)
    else:
      print("Ok deleting Slack message.")      
    time.sleep(1)    

In [5]:
messageList = getMessageList()
if messageList is not None:
   #logger(_LOG_INFO, messageList['matches'])
   for n in range(0, len(messageList['matches'])):
      message = messageList['matches'][n]
      channelname = message['channel']['name']
      #logger(_LOG_INFO, "{}".format(channelname))
      #if channelname == "rfsurvey":
      channelid = message['channel']['id']
      ts = message['ts']
      logger(_LOG_INFO, "deleting {} {} {}".format(channelname, channelid, ts))
      deleteMessage(channelid, ts)
logger(_LOG_INFO, "ScrubChannel Done!")

Ok searching Slack workspace.
deleting infrastructure C013YFJF276 1593960310.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593959410.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593957610.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593958511.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593955810.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593950410.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593949510.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593945910.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593954010.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593948611.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593956711.000100
Ok deleting Slack message.
deleting infrastructure C013YFJF276 1593945011.000100
Ok deleting Slack message